In [ ]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader


## Load data

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.vectorstores.pinecone import Pinecone
from langchain.chains import RetrievalQA
from langchain import OpenAI
from langchain_openai import ChatOpenAI
from cleantext import clean
import os
import nltk
import pinecone
import openai

In [ ]:
from pinecone import Pinecone

# configure client
pc = Pinecone()

In [ ]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'
project_name = "Serverless"
spec = ServerlessSpec(cloud=cloud, region=region)
index_name = "yoga-kb"

In [ ]:
pc.list_indexes()

In [ ]:
index = pc.Index(index_name, project_name)
index.describe_index_stats()

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)


In [ ]:
vectorstore_from_docs = PineconeVectorStore.from_documents(
        split_texts,
        index_name=index_name,
        embedding=embeddings
    )

In [ ]:
query = "What is minimum viable product?"
vectorstore.similarity_search(query)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
# completion llm
llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.0
)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)
qa.run(query)

In [ ]:
initial_prompt = "You are a researcher who is going to search the web links, summarize them and share insights as asked"

In [ ]:
query = "What is the primary goal of minimum viable product?"
result = qa({"query": query, "prompt": initial_prompt})

In [ ]:
print(result['result'])